In [1]:
%matplotlib inline
import tensorflow as tf
from vahun.Text import Text
import numpy as np
from vahun.tools import Timer
from vahun.tools import explog
from vahun.autoencoder import Autoencoder_ffnn
from vahun.genetic import evolution
from vahun.genetic import experiment
from vahun.tools import show_performance

encode=180

popsize=1


config = tf.ConfigProto()
config.gpu_options.allow_growth = True

corpus=Text(corpus_path='/mnt/store/velkey/mnsz2/mnsz2.100M.freqs.hunfiltered_lowered',size=1000000)

logger=explog(encoder_type="hunfiltered_lowered_uniq_"+str(encode),
              encoding_dim=encode,feature_len=10,
              lang="Hun",unique_words=len(set(corpus.wordlist)),
              name="autoencoder_lowered_"+str(encode),population_size=popsize,
              words=len(corpus.wordlist))

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

exp=experiment(encoded_width=10,layermax=10,layermin=2,maxw=10,minw=3,out_dim=corpus.abc*10)

exp.weights=[encode, len(corpus.abc)*10]
exp.len=len(exp.weights)

In [2]:
encoder=Autoencoder_ffnn(experiment=exp,
                 logger=logger,tf_session=sess,
                 inputdim=len(corpus.abc)*10,
                 layerlist=exp.weights,
                 encode_index=int(exp.len/2),corpus=corpus,
                 optimizer =tf.train.AdamOptimizer(learning_rate = 0.001),
                 nonlinear=tf.sigmoid,
                 charnum=len(corpus.abc))

In [3]:
encoder.train(corpus.x_train,corpus.x_valid,corpus.x_test,512,30)

In [4]:
stds=show_performance(encoder,corpus.x_test,corpus,10,printer=True,inputfsize=len(corpus.abc))

   tömzsin 	    tömzsin 	 0
 reményünk 	  reményünk 	 0
sűrítménye 	 szrétménye 	 2
foglyá-nak 	 foglyájnak 	 1
      samu 	       sagb 	 2
  napszagú 	   napszagy 	 1
moderatúra 	 moderatóra 	 1
amarillisz 	 emarillisz 	 1
híradásból 	 haradásból 	 1
  nyakéked 	   nyakéked 	 0

Accuracy on data:  95.046272145907 %
average Levenshtein distance:  0.495372785409


In [5]:
std=show_performance(encoder,["eh","kecske","kutya","aytuk","macska","árvíztűrő","fúró","kacsa","a","és"],corpus,printer=True,inputfsize=len(corpus.abc))

        eh 	         er 	 1
    kecske 	     kecske 	 0
     kutya 	      kutya 	 0
     aytuk 	      aytuk 	 0
    macska 	     macska 	 0
 árvíztűrő 	  árvöztörő 	 2
      fúró 	        óró 	 2
     kacsa 	      kapsa 	 1
         a 	         za 	 1
        és 	         és 	 0

Accuracy on data:  93.0 %
average Levenshtein distance:  0.7


In [6]:
def decode_critical(lista,enc=180):
    for POS in lista:
        encoded=np.ones(enc)*-10
        encoded[POS]=10
        a=encoder.decode([encoded])[0].reshape([10,len(corpus.abc)])
        b=corp.defeaturize_data_charlevel_onehot([a])
        print(b)

In [7]:
import heapq
topstd=heapq.nlargest(6, range(len(stds)), stds.__getitem__)
big_stuff=[]
for i in range(50):
    big_stuff.append([])
    for topind in topstd:
        a=encoder.encode([corpus.x_test[i]])
        a[0][topind]=2
        b=encoder.decode([a[0]])[0].reshape([10,len(corpus.abc)])
        c=corpus.defeaturize_data_charlevel_onehot([b])
        big_stuff[i].append(c)
for row in big_stuff:
    print(row)

[['   temzsin'], ['ts tömzsin'], ['   témzsin'], ['   tömzsia'], ['    ömzsnn'], ['   tamzsin']]
[[' reményenk'], ['treménzlnk'], [' r ménymnk'], [' reményynk'], [' reményrnk'], [' reményank']]
[['szr tménee'], ['szrntménle'], ['  r tménve'], ['sárotménóe'], ['szr tménne'], ['sarltménne']]
[['fogleáenak'], ['foelyáznak'], ['foglyálnak'], ['fogliáonak'], ['foglyánnak'], ['foglyáznak']]
[['      saek'], ['ts    zalt'], ['      savó'], ['      saóa'], ['      nand'], ['      sant']]
[['  napezaek'], ['tenapszaőt'], ['  napszavó'], ['  napszaga'], ['  napszani'], ['  napszant']]
[['moderetera'], ['moderatlrt'], ['moderatmra'], ['moderatyra'], [' od ratrna'], ['moderatara']]
[[' marielesz'], ['kmarilzist'], [' marillisz'], [' marillisa'], [' ma isnisz'], ['em rillisz']]
[[' áradásból'], ['heradázbót'], ['  radásból'], [' áradásból'], [' oradásból'], ['haradásból']]
[['  nyakéked'], ['ksnyakéked'], ['  nyakéked'], ['  nyakéked'], ['  n akéked'], ['  nyakéked']]
[['      eeea'], ['ts    zllt'